# **Criando o Banco de Dados**

Notebook com os comandos necessários para criar um banco SQL no SQLite e carregar os dados dos arquivos fornecidos nele.

**Arquivos fornecidos:**
- *train_houses.xlsx*
- *test_houses.xlsx* --> sem valores na coluna de preço (target).

## Bibliotecas

In [1]:
import pandas as pd
import sqlite3
import re

## Funções

In [2]:
# Função para formatar os arquivos brutos
def format_table(df):
    # Passo 1: Substituir vírgulas seguidas de espaços por um espaço
    df = df.applymap(lambda x: re.sub(r',\s', ' ', x) if isinstance(x, str) else x)

    # Passo 2: Substituir vírgulas que não estão dentro de aspas duplas e seguidas por caracteres não vazios por ';'
    df = df.applymap(lambda x: re.sub(r',(?=(?:[^"]*"[^"]*")*[^"]*$)(?!\s)', ';', x) if isinstance(x, str) else x)

    # Etapa 3: Separar os valores por ponto e vírgula em colunas
    # Unir todas as colunas em uma única coluna para processar
    df_combined = df.apply(lambda row: ';'.join(row.dropna().astype(str)), axis=1)
    data = df_combined.str.split(';', expand=True)

    # Etapa 4: Configurar a primeira linha como cabeçalho
    columns = data.iloc[0]
    data = data[1:]
    data.columns = columns

    return data

## Carregando Dados

In [3]:
df_train = pd.read_excel('data/train_houses.xlsx', header=None)
df_train.head()

,0
0,"Type,Region,MunicipalityCode,Prefecture,Munici..."
1,"Pre-owned Condominiums, etc.,,13103,Tokyo,Mina..."
2,"Residential Land(Land and Building),Residentia..."
3,"Residential Land(Land Only),Residential Area,1..."
4,"Pre-owned Condominiums, etc.,,13208,Tokyo,Chof..."


In [14]:
df1 = pd.read_excel('data/train_houses.xlsx', header=None)

import re
import pandas as pd

# Função para formatar os arquivos brutos
def format_table(df):
    # Passo 1: Substituir vírgulas seguidas de espaços por um espaço
    df = df.apply(lambda col: col.map(lambda x: re.sub(r',\s', ' ', x) if isinstance(x, str) else x))

    # Passo 2: Substituir vírgulas que não estão dentro de aspas duplas e seguidas por caracteres não vazios por ';'
    df = df.apply(lambda col: col.map(lambda x: re.sub(r',(?=(?:[^"]*"[^"]*")*[^"]*$)(?!\s)', ';', x) if isinstance(x, str) else x))

    # Etapa 3: Separar os valores por ponto e vírgula em colunas
    # Unir todas as colunas em uma única coluna para processar
    df_combined = df.apply(lambda row: ';'.join(row.dropna().astype(str)), axis=1)
    data = df_combined.str.split(';', expand=True)

    # Etapa 4: Configurar a primeira linha como cabeçalho
    columns = data.iloc[0]
    data = data[1:]
    data.columns = columns

    return data

# Testar a função
df_formatted = format_table(df1)

print("\nDataFrame Formatado:")
df_formatted.sample(5)



DataFrame Formatado:


,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,MaxTimeToNearestStation,...,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks,TradePrice
238751,Residential Land(Land Only),Residential Area,13113,Tokyo,Shibuya Ward,Sasazuka,Sasazuka,8,8.0,8.0,...,3.5,Category I Residential Zone,60.0,300.0,2nd quarter 2012,2012,2,,,19000000
205974,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Kudamminami,Ichigaya,6,6.0,6.0,...,11.0,Commercial Zone,80.0,500.0,2nd quarter 2018,2018,2,,,340000000
163810,Residential Land(Land Only),Residential Area,13119,Tokyo,Itabashi Ward,Akatsuka,Narimasu,13,13.0,13.0,...,4.0,Category I Exclusively Low-story Residential Zone,40.0,80.0,4th quarter 2007,2007,4,,,25000000
64462,Pre-owned Condominiums etc.,,13108,Tokyo,Koto Ward,Kitasuna,Nishiojima,13,13.0,13.0,...,,Neighborhood Commercial Zone,80.0,300.0,2nd quarter 2019,2019,2,Not yet,,8500000
202137,Residential Land(Land Only),Residential Area,13209,Tokyo,Machida City,Kanamori,Machida,24,24.0,24.0,...,6.0,Category I Exclusively Low-story Residential Zone,40.0,80.0,2nd quarter 2012,2012,2,,,25000000


In [4]:
df_test = pd.read_excel('data/test_houses.xlsx', header=None)
df_test.head()

,0
0,"Type,Region,MunicipalityCode,Prefecture,Munici..."
1,"Pre-owned Condominiums, etc.,,13103,Tokyo,Mina..."
2,"Pre-owned Condominiums, etc.,,13110,Tokyo,Megu..."
3,"Pre-owned Condominiums, etc.,,13112,Tokyo,Seta..."
4,"Pre-owned Condominiums, etc.,,13121,Tokyo,Adac..."


## Formatando Dados

Nem todas as vírgulas do dataset são delimitadores de colunas:
- vírgulas seguidas de espaço: *exemplo, etc*
- vírgulas dentro de uma expressão com aspas duplas: *"Cidade, Província"*

In [6]:
# Aplicar a função de formatação em df_train
df_train_formatted = format_table(df_train)

print("DataFrame de TREINO Corrigido:")
df_train_formatted.sample(5)

/tmp/ipykernel_845/320415916.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: re.sub(r',\s', ' ', x) if isinstance(x, str) else x)
/tmp/ipykernel_845/320415916.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: re.sub(r',(?=(?:[^"]*"[^"]*")*[^"]*$)(?!\s)', ';', x) if isinstance(x, str) else x)


DataFrame de TREINO Corrigido:


,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,MaxTimeToNearestStation,...,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks,TradePrice
69395,Residential Land(Land and Building),Residential Area,13228,Tokyo,Akiruno City,Hikida,Musashihikida,14,14.0,14.0,...,6.0,Category I Exclusively Low-story Residential Zone,40.0,80.0,2nd quarter 2012,2012,2,,,4900000
309423,Residential Land(Land Only),Residential Area,13120,Tokyo,Nerima Ward,Higashioizumi,Oizumigakuen,8,8.0,8.0,...,6.0,Category I Residential Zone,60.0,200.0,2nd quarter 2015,2015,2,,,29000000
138235,Pre-owned Condominiums etc.,,13115,Tokyo,Suginami Ward,Koenjikita,Koenji,4,4.0,4.0,...,,Category I Exclusively Medium-high Residential...,60.0,200.0,1st quarter 2018,2018,1,Not yet,,53000000
280026,Residential Land(Land and Building),Residential Area,13106,Tokyo,Taito Ward,Minowa,Minowa,3,3.0,3.0,...,4.0,Commercial Zone,80.0,400.0,2nd quarter 2013,2013,2,,,52000000
177154,Residential Land(Land Only),Residential Area,13120,Tokyo,Nerima Ward,Nakamuraminami,Nerima,15,15.0,15.0,...,11.0,Neighborhood Commercial Zone,80.0,200.0,3rd quarter 2014,2014,3,,,46000000


In [7]:
# Aplicar a função de formatação em df_test
df_test_formatted = format_table(df_test)

print("DataFrame de TESTE Corrigido:")
df_test_formatted.sample(5)

/tmp/ipykernel_845/320415916.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: re.sub(r',\s', ' ', x) if isinstance(x, str) else x)
/tmp/ipykernel_845/320415916.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: re.sub(r',(?=(?:[^"]*"[^"]*")*[^"]*$)(?!\s)', ';', x) if isinstance(x, str) else x)


DataFrame de TESTE Corrigido:


,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,MaxTimeToNearestStation,...,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks,TradePrice
51171,Residential Land(Land and Building),Residential Area,13110,Tokyo,Meguro Ward,Senzoku,Senzoku,3,3,3,...,5.3,Category I Exclusively Low-story Residential Zone,50,100,3rd quarter 2019,2019,3,,,
26128,Residential Land(Land and Building),Commercial Area,13123,Tokyo,Edogawa Ward,Matsushima,Shinkoiwa,7,7,7,...,7.2,Commercial Zone,80,400,2nd quarter 2018,2018,2,,Dealings including private road,
17839,Residential Land(Land Only),Residential Area,13209,Tokyo,Machida City,Tokiwamachi,Fuchinobe,30-60minutes,30,60,...,5,Category I Exclusively Low-story Residential Zone,40,80,2nd quarter 2010,2010,2,,,
64298,Residential Land(Land and Building),Residential Area,13220,Tokyo,Higashiyamato City,Zoshiki,Kamikitadai,14,14,14,...,4.5,Category I Exclusively Low-story Residential Zone,40,80,2nd quarter 2014,2014,2,,,
767,Residential Land(Land and Building),Residential Area,13109,Tokyo,Shinagawa Ward,Oi,Oimachi,10,10,10,...,4,Category I Residential Zone,60,200,2nd quarter 2017,2017,2,,Dealings including private road,


## Criando conexão ao banco SQL

In [8]:
# Conectar ao banco de dados SQLite (ou criar se não existir)
conn = sqlite3.connect('data/house_prices.db')

## Carregando dados no banco SQL

In [9]:
# Salvando no banco de dados como tabela 'df_train'
df_train_formatted.to_sql('df_train', conn, if_exists='replace', index=False)

# Salvando no banco de dados como tabela 'df_test'
df_test_formatted.to_sql('df_test', conn, if_exists='replace', index=False)

81314

## Checando tabelas do banco SQL

In [10]:
# Consultando os dados do banco para verificar se estão corretos
query_train = "SELECT * FROM df_train LIMIT 5"
df_train_check = pd.read_sql(query_train, conn)

print("Dados da tabela df_train:")
df_train_check

Dados da tabela df_train:


,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,MaxTimeToNearestStation,...,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks,TradePrice
0,Pre-owned Condominiums etc.,,13103,Tokyo,Minato Ward,Kaigan,Takeshiba,1,1.0,1.0,...,,Quasi-industrial Zone,60.0,400.0,1st quarter 2011,2011,1,Done,,24000000
1,Residential Land(Land and Building),Residential Area,13120,Tokyo,Nerima Ward,Nishiki,Kamiitabashi,15,15.0,15.0,...,4.0,Category I Exclusively Low-story Residential Zone,60.0,200.0,3rd quarter 2013,2013,3,,Dealings including private road,51000000
2,Residential Land(Land Only),Residential Area,13201,Tokyo,Hachioji City,Shimoongatamachi,Takao (Tokyo),1H-1H30,60.0,90.0,...,4.5,Category I Exclusively Low-story Residential Zone,40.0,80.0,4th quarter 2007,2007,4,,,14000000
3,Pre-owned Condominiums etc.,,13208,Tokyo,Chofu City,Kamiishiwara,Nishichofu,16,16.0,16.0,...,,Quasi-industrial Zone,60.0,200.0,2nd quarter 2015,2015,2,Not yet,,23000000
4,Residential Land(Land Only),Residential Area,13117,Tokyo,Kita Ward,Shimo,Shimo,6,6.0,6.0,...,4.5,Category I Exclusively Medium-high Residential...,60.0,200.0,4th quarter 2015,2015,4,,,33000000


In [11]:
# Consultando os dados do banco para verificar se estão corretos
query_test = "SELECT * FROM df_test LIMIT 5"
df_test_check = pd.read_sql(query_test, conn)

print("Dados da tabela df_test:")
df_test_check

Dados da tabela df_test:


,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,MaxTimeToNearestStation,...,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks,TradePrice
0,Pre-owned Condominiums etc.,,13103,Tokyo,Minato Ward,Toranomon,Kamiyacho,4,4,4,...,,Commercial Zone,80,500,3rd quarter 2016,2016,3,Not yet,,
1,Pre-owned Condominiums etc.,,13110,Tokyo,Meguro Ward,Higashiyama,Ikejiriohashi,7,7,7,...,,Category I Residential Zone,60,300,3rd quarter 2012,2012,3,,,
2,Pre-owned Condominiums etc.,,13112,Tokyo,Setagaya Ward,Kitakarasuyama,Chitosekarasuyama,25,25,25,...,,Category I Exclusively Low-story Residential Zone,50,100,4th quarter 2015,2015,4,Done,,
3,Pre-owned Condominiums etc.,,13121,Tokyo,Adachi Ward,Ayase,Ayase,4,4,4,...,,Commercial Zone,80,500,2nd quarter 2017,2017,2,Done,,
4,Residential Land(Land and Building),Residential Area,13107,Tokyo,Sumida Ward,Honjo,Honjoazumabashi,7,7,7,...,6,Neighborhood Commercial Zone,80,300,3rd quarter 2016,2016,3,,,


In [12]:
# Fechar a conexão com o banco de dados
conn.close()

print("Dados carregados com sucesso no banco de dados SQLite.")

Dados carregados com sucesso no banco de dados SQLite.
